<a href="https://colab.research.google.com/github/mbayekebe/Coursera_Capstone/blob/master/TorontoVenueCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Producing Toronto maps with Venues and the tables for Venue categories

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [0]:
#read csv file of geocoded neighborhood in Toronto
neighborhoods = pd.read_csv('GeocodedPostalCodesToronto.csv', index_col=0)
# torontodf

In [0]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Toronto are 43.653963, -79.387207.


In [0]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [0]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=500, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [0]:
limit = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'U4WSHP0EGOQG525TYOCH0Y4MQYQWPBOSONLWBLIB3WDM5E4Z'
CLIENT_SECRET = 'E4VNR2QAD4MNVTCJR3NNS3RPUAVFB20FUZE3WDQIWJS1YZGT'
VERSION = '20181020'

In [0]:
# Use category id 4d4b7104d754a06370d81259 Arts & Entertainmemnt
toronto_venues_arts = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7104d754a06370d81259')
toronto_venues_arts.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
1,"Harbourfront,Regent Park",43.654260,-79.360636,Orphan Black Police Station,43.655757,-79.358012,Street Art
2,"Harbourfront,Regent Park",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.654260,-79.360636,Archive - Gallery and Art Library,43.652790,-79.363313,Art Gallery
4,"Harbourfront,Regent Park",43.654260,-79.360636,Boat Rocker,43.653031,-79.363335,General Entertainment


In [0]:
toronto_venues_arts.shape

(1154, 7)

In [0]:
toronto_venues_arts.to_csv('TorontoVenueArts.csv')

In [0]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [0]:
 map_toronto_arts = folium.Map(location=[latitude, longitude], zoom_start=12)
 addToMap(toronto_venues_arts, 'red', map_toronto_arts)
 map_toronto_arts

In [0]:
# Use category id 4d4b7105d754a06372d81259 College & University
toronto_venues_colu = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06372d81259')
toronto_venues_colu.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Ranchdale,43.752319,-79.323448,College Classroom
1,Victoria Village,43.725882,-79.315572,Toronto Fire Services Operations Training Centre,43.723972,-79.317126,Trade School
2,"Harbourfront,Regent Park",43.654260,-79.360636,The George,43.653246,-79.357222,College Residence Hall
3,"Harbourfront,Regent Park",43.654260,-79.360636,Charles MacPherson Associates,43.654681,-79.359209,Trade School
4,"Harbourfront,Regent Park",43.654260,-79.360636,George Brown College - School of ESL,43.651872,-79.365580,College Academic Building


In [0]:
toronto_venues_colu.shape

(682, 7)

In [0]:
toronto_venues_colu.to_csv('TorontoVenueColU.csv')

In [0]:
# Use category id 4d4b7105d754a06374d81259 Food
toronto_venues_food = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06374d81259')
toronto_venues_food.head()
toronto_venues_food.shape


(2322, 7)

In [0]:
toronto_venues_food.to_csv('TorontoVenueFood.csv')

In [0]:
# Use category id 4d4b7105d754a06375d81259 Professional & Other
toronto_venues_prother = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06375d81259')
toronto_venues_prother.head()
toronto_venues_prother.shape

(2565, 7)

In [0]:
toronto_venues_prother.to_csv('TorontoVenueProf.csv')

In [0]:
# Use category id 4d4b7105d754a06376d81259 Nighlife
toronto_venues_nighlife = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06376d81259')
toronto_venues_nighlife.head()
toronto_venues_nighlife.shape

(1103, 7)

In [0]:
toronto_venues_nighlife.to_csv('TorontoVenueNightlife.csv')

In [0]:
# Use category id 4d4b7105d754a06377d81259 Outdoor & Recreation
toronto_venues_outdoors = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06377d81259')
toronto_venues_outdoors.head()
toronto_venues_outdoors.shape

(1604, 7)

In [0]:
toronto_venues_outdoors.to_csv('TorontoVenueOutdoors.csv')

In [0]:
# Use category id 4d4b7105d754a06378d81259 Shop & Service
toronto_venues_shopserv = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06378d81259')
toronto_venues_shopserv.head()
toronto_venues_shopserv.shape

(2607, 7)

In [0]:
toronto_venues_shopserv.to_csv('TorontoVenueShopServ.csv')

In [0]:
# Use category id 4d4b7105d754a06379d81259 Travel & Transport
toronto_venues_travel = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4d4b7105d754a06379d81259')
toronto_venues_travel.head()
toronto_venues_travel.shape

(1090, 7)

In [0]:
toronto_venues_travel.to_csv('TorontoVenueTravels.csv')

In [0]:
# Use category id 4e67e38e036454776db1fb3a Residence
toronto_venues_residence = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4e67e38e036454776db1fb3a')
toronto_venues_residence.head()
toronto_venues_residence.shape

(1101, 7)

In [0]:
toronto_venues_residence.to_csv('TorontoVenueResidence.csv')

In [0]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [0]:
#read csv file of geocoded neighborhood in Toronto
toronto_venues_arts = pd.read_csv('TorontoVenueArts.csv', index_col=0)
toronto_venues_colu = pd.read_csv('TorontoVenueColU.csv', index_col=0)
toronto_venues_food = pd.read_csv('TorontoVenueFood.csv', index_col=0)
toronto_venues_prother = pd.read_csv('TorontoVenueProf.csv', index_col=0)
toronto_venues_nightlife = pd.read_csv('TorontoVenueNightlife.csv', index_col=0)
toronto_venues_outdoors = pd.read_csv('TorontoVenueOutdoors.csv', index_col=0)
toronto_venues_shopserv = pd.read_csv('TorontoVenueShopServ.csv', index_col=0)
toronto_venues_travel = pd.read_csv('TorontoVenueTravels.csv', index_col=0)
toronto_venues_residence = pd.read_csv('TorontoVenueResidence.csv', index_col=0)

In [0]:
df_data = neighborhoods.copy()
# df_data.rename(columns={'neighborhoods':'Neighborhood'}, inplace=True)
addColumn(df_data, 'Arts', toronto_venues_arts)
addColumn(df_data, 'CollegeUniversity', toronto_venues_colu)
addColumn(df_data, 'Food', toronto_venues_food)
addColumn(df_data, 'Professional', toronto_venues_prother)
addColumn(df_data, 'Nightlife', toronto_venues_nightlife)
addColumn(df_data, 'Outdoor', toronto_venues_outdoors)
addColumn(df_data, 'ShopServices', toronto_venues_shopserv)
addColumn(df_data, 'TravelTransport', toronto_venues_travel)
addColumn(df_data, 'Residence', toronto_venues_residence)
df_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Arts,CollegeUniversity,Food,Professional,Nightlife,Outdoor,ShopServices,TravelTransport,Residence
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,1.0,1.0,9.0,0.0,3.0,4.0,4.0,4.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,1.0,5.0,19.0,0.0,5.0,15.0,4.0,7.0
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,50.0,8.0,50.0,49.0,27.0,47.0,50.0,38.0,35.0
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,4.0,3.0,12.0,28.0,2.0,6.0,39.0,1.0,0.0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,45.0,48.0,50.0,50.0,41.0,45.0,50.0,28.0,49.0
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,0.0,0.0,2.0,2.0,0.0,3.0,1.0,3.0,1.0
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,0.0,2.0,20.0,0.0,3.0,12.0,0.0,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,5.0,8.0,43.0,0.0,13.0,8.0,1.0,0.0
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,2.0,0.0,22.0,18.0,4.0,7.0,26.0,7.0,2.0
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,47.0,49.0,50.0,50.0,47.0,46.0,50.0,44.0,49.0


In [0]:
df_data.to_csv('TorontoVenueCategoriesGrouped.csv')